In [8]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession,Window

In [2]:
spark=SparkSession.builder.\
    appName("Practice").\
        getOrCreate()
    

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/26 11:57:16 INFO SparkEnv: Registering MapOutputTracker
24/05/26 11:57:16 INFO SparkEnv: Registering BlockManagerMaster
24/05/26 11:57:16 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/05/26 11:57:16 INFO SparkEnv: Registering OutputCommitCoordinator


In [3]:
employees_df =spark.read.format("csv").\
    option("header","true").\
        option("inferschema","true").\
            option("mode","PERMISSIVE").\
                load("/user/sovik/employees.csv")

In [15]:
employees_df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Nominee: string (nullable = true)



In [16]:
employees_df.createOrReplaceTempView("Emp_temp_view")

In [18]:
spark.sql("""
          
          Select * from Emp_temp_view where Country ='Canada'
          
          """).show(truncate=False)

+---+---------------+------+-------+--------------------------+---------+
|ID |Name           |Salary|Country|Address                   |Nominee  |
+---+---------------+------+-------+--------------------------+---------+
|2  |Jane Smith     |60000 |Canada |456 Maple St Oaktown CA   |Nominee2 |
|6  |Ava Martinez   |75000 |Canada |303 Birch St Mapletown CA |Nominee6 |
|10 |Sophia Taylor  |80000 |Canada |707 Cherry St Hillside CA |Nominee10|
|14 |Mia Harris     |78000 |Canada |222 Maple St Mountview CA |Nominee14|
|18 |Amelia Hall    |85000 |Canada |666 Spruce St Highland CA |Nominee18|
|22 |Charlotte Scott|90000 |Canada |123 Main St Snowtown CA   |Nominee22|
|26 |Sophia Nelson  |74000 |Canada |567 Maple St Northwood CA |Nominee26|
|30 |Ella Roberts   |80000 |Canada |901 Spruce St Woodlands CA|Nominee30|
+---+---------------+------+-------+--------------------------+---------+



In [19]:
employees_df.select(col("ID").alias("Employee_id"),employees_df.Name.alias("Employee_name")).show()

+-----------+---------------+
|Employee_id|  Employee_name|
+-----------+---------------+
|          1|       John Doe|
|          2|     Jane Smith|
|          3|      Jim Brown|
|          4|     Emma Davis|
|          5|   Liam Johnson|
|          6|   Ava Martinez|
|          7|    Noah Miller|
|          8|     Mia Wilson|
|          9|  William Moore|
|         10|  Sophia Taylor|
|         11| Mason Anderson|
|         12|Isabella Thomas|
|         13|    Jacob White|
|         14|     Mia Harris|
|         15|    Ethan Clark|
|         16|   Olivia Lewis|
|         17|   Lucas Walker|
|         18|    Amelia Hall|
|         19|    Henry Young|
|         20|       Ava King|
+-----------+---------------+
only showing top 20 rows



lets add quebecoise as name to whoever is living in Maple St in Canada

In [25]:
quebeqoise_employee_df=employees_df.select(col("*"),lit("Quebeqoise").alias("quebeqise_name")).where("address like '%Maple St%' and country = 'Canada'")

In [27]:
quebeqoise_employee_df.show(truncate=False)

+---+-------------+------+-------+-------------------------+---------+--------------+
|ID |Name         |Salary|Country|Address                  |Nominee  |quebeqise_name|
+---+-------------+------+-------+-------------------------+---------+--------------+
|2  |Jane Smith   |60000 |Canada |456 Maple St Oaktown CA  |Nominee2 |Quebeqoise    |
|14 |Mia Harris   |78000 |Canada |222 Maple St Mountview CA|Nominee14|Quebeqoise    |
|26 |Sophia Nelson|74000 |Canada |567 Maple St Northwood CA|Nominee26|Quebeqoise    |
+---+-------------+------+-------+-------------------------+---------+--------------+



In [33]:
employee_df_with_quebeqoise = quebeqoise_employee_df.withColumn("any_name",when(concat(col("Name"), col("quebeqoise_name")).alias("real_quebeqoise_name"))

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `quebeqoise_name` cannot be resolved. Did you mean one of the following? [`quebeqise_name`, `Name`, `Nominee`, `Address`, `Country`].;
'Project [ID#105, Name#106, Salary#107, Country#108, Address#109, Nominee#110, quebeqise_name#309, concat(Name#106, 'quebeqoise_name) AS real_quebeqoise_name#389]
+- Filter (address#109 LIKE %Maple St% AND (country#108 = Canada))
   +- Project [ID#105, Name#106, Salary#107, Country#108, Address#109, Nominee#110, Quebeqoise AS quebeqise_name#309]
      +- Relation [ID#105,Name#106,Salary#107,Country#108,Address#109,Nominee#110] csv


In [4]:
emp_data = [(1,'manish',50000,'IT','m'),
(2,'vikash',60000,'sales','m'),
(3,'raushan',70000,'marketing','m'),
(4,'mukesh',80000,'IT','m'),
(5,'priti',90000,'sales','f'),
(6,'nikita',45000,'marketing','f'),
(7,'ragini',55000,'marketing','f'),
(8,'rashi',100000,'IT','f'),
(9,'aditya',65000,'IT','m'),
(10,'rahul',50000,'marketing','m'),
(11,'rakhi',50000,'IT','f'),
(12,'akhilesh',90000,'sales','m')]

In [5]:
my_schema=['id','name','sal','dept','sex']

In [6]:
my_emp=spark.createDataFrame(emp_data,schema=my_schema)

In [7]:
my_emp.show()

+---+--------+------+---------+---+
| id|    name|   sal|     dept|sex|
+---+--------+------+---------+---+
|  1|  manish| 50000|       IT|  m|
|  2|  vikash| 60000|    sales|  m|
|  3| raushan| 70000|marketing|  m|
|  4|  mukesh| 80000|       IT|  m|
|  5|   priti| 90000|    sales|  f|
|  6|  nikita| 45000|marketing|  f|
|  7|  ragini| 55000|marketing|  f|
|  8|   rashi|100000|       IT|  f|
|  9|  aditya| 65000|       IT|  m|
| 10|   rahul| 50000|marketing|  m|
| 11|   rakhi| 50000|       IT|  f|
| 12|akhilesh| 90000|    sales|  m|
+---+--------+------+---------+---+



In [9]:
window_spec=Window.partitionBy(col("dept")).orderBy(col("sal").desc())

In [19]:
window_spec_totsal=Window.partitionBy(col("dept"))

In [15]:
intermediate_df=my_emp.withColumn("rank_sal_by_dept",row_number().over(window_spec) )

In [20]:
intermediate_df_tot_sal=my_emp.withColumn("total_sal",sum(col("sal")).over(window_spec_totsal))

In [21]:
intermediate_df_tot_sal.show()

+---+--------+------+---------+---+---------+
| id|    name|   sal|     dept|sex|total_sal|
+---+--------+------+---------+---+---------+
|  1|  manish| 50000|       IT|  m|   345000|
|  4|  mukesh| 80000|       IT|  m|   345000|
|  8|   rashi|100000|       IT|  f|   345000|
|  9|  aditya| 65000|       IT|  m|   345000|
| 11|   rakhi| 50000|       IT|  f|   345000|
|  3| raushan| 70000|marketing|  m|   220000|
|  6|  nikita| 45000|marketing|  f|   220000|
|  7|  ragini| 55000|marketing|  f|   220000|
| 10|   rahul| 50000|marketing|  m|   220000|
|  2|  vikash| 60000|    sales|  m|   240000|
|  5|   priti| 90000|    sales|  f|   240000|
| 12|akhilesh| 90000|    sales|  m|   240000|
+---+--------+------+---------+---+---------+



In [17]:
intermediate_df.filter(col("rank_sal_by_dept")<=2).show()

+---+--------+------+---------+---+----------------+
| id|    name|   sal|     dept|sex|rank_sal_by_dept|
+---+--------+------+---------+---+----------------+
|  8|   rashi|100000|       IT|  f|               1|
|  4|  mukesh| 80000|       IT|  m|               2|
|  3| raushan| 70000|marketing|  m|               1|
|  7|  ragini| 55000|marketing|  f|               2|
|  5|   priti| 90000|    sales|  f|               1|
| 12|akhilesh| 90000|    sales|  m|               2|
+---+--------+------+---------+---+----------------+



In [22]:
product_data = [
(1,"iphone","01-01-2023",1500000),
(2,"samsung","01-01-2023",1100000),
(3,"oneplus","01-01-2023",1100000),
(1,"iphone","01-02-2023",1300000),
(2,"samsung","01-02-2023",1120000),
(3,"oneplus","01-02-2023",1120000),
(1,"iphone","01-03-2023",1600000),
(2,"samsung","01-03-2023",1080000),
(3,"oneplus","01-03-2023",1160000),
(1,"iphone","01-04-2023",1700000),
(2,"samsung","01-04-2023",1800000),
(3,"oneplus","01-04-2023",1170000),
(1,"iphone","01-05-2023",1200000),
(2,"samsung","01-05-2023",980000),
(3,"oneplus","01-05-2023",1175000),
(1,"iphone","01-06-2023",1100000),
(2,"samsung","01-06-2023",1100000),
(3,"oneplus","01-06-2023",1200000)
]

product_schema=['product_id',"product_name","sale_date","sale_amount"]

product_df=spark.createDataFrame(product_data,product_schema)

product_df.show()

+----------+------------+----------+-----------+
|product_id|product_name| sale_date|sale_amount|
+----------+------------+----------+-----------+
|         1|      iphone|01-01-2023|    1500000|
|         2|     samsung|01-01-2023|    1100000|
|         3|     oneplus|01-01-2023|    1100000|
|         1|      iphone|01-02-2023|    1300000|
|         2|     samsung|01-02-2023|    1120000|
|         3|     oneplus|01-02-2023|    1120000|
|         1|      iphone|01-03-2023|    1600000|
|         2|     samsung|01-03-2023|    1080000|
|         3|     oneplus|01-03-2023|    1160000|
|         1|      iphone|01-04-2023|    1700000|
|         2|     samsung|01-04-2023|    1800000|
|         3|     oneplus|01-04-2023|    1170000|
|         1|      iphone|01-05-2023|    1200000|
|         2|     samsung|01-05-2023|     980000|
|         3|     oneplus|01-05-2023|    1175000|
|         1|      iphone|01-06-2023|    1100000|
|         2|     samsung|01-06-2023|    1100000|
|         3|     one

What is the percentage of loss or gain depending on previous month sale


In [23]:
product_spec=Window.partitionBy(col("product_name")).orderBy("sale_date")

In [35]:
product_df_with_lag=product_df.withColumn("previous_month_sale",lag(col("sale_amount"),1,0).over(product_spec))
#product_df_with_lag_final=product_df_with_lag.withColumn("previous_month_sale",when(isnull(col("previous_month_sale")),0).otherwise(col("previous_month_sale")) )
product_df_with_lag.show()

+----------+------------+----------+-----------+-------------------+
|product_id|product_name| sale_date|sale_amount|previous_month_sale|
+----------+------------+----------+-----------+-------------------+
|         1|      iphone|01-01-2023|    1500000|                  0|
|         1|      iphone|01-02-2023|    1300000|            1500000|
|         1|      iphone|01-03-2023|    1600000|            1300000|
|         1|      iphone|01-04-2023|    1700000|            1600000|
|         1|      iphone|01-05-2023|    1200000|            1700000|
|         1|      iphone|01-06-2023|    1100000|            1200000|
|         3|     oneplus|01-01-2023|    1100000|                  0|
|         3|     oneplus|01-02-2023|    1120000|            1100000|
|         3|     oneplus|01-03-2023|    1160000|            1120000|
|         3|     oneplus|01-04-2023|    1170000|            1160000|
|         3|     oneplus|01-05-2023|    1175000|            1170000|
|         3|     oneplus|01-06-202

In [40]:
product_df_with_lag_final_vf = product_df_with_lag.withColumn(
    "profit_by_over_previous_month",
    round(
        when(col("previous_month_sale") != 0, 
             ((col("sale_amount") - col("previous_month_sale")) * 100) / col("previous_month_sale"))
        .otherwise("NA"), 2
    )
)
product_df_with_lag_final_vf2 = product_df_with_lag_final_vf.withColumn(
    "profit_by_over_previous_month",
    when(col("profit_by_over_previous_month").isNull(), "NA")
    .otherwise(col("profit_by_over_previous_month"))
)

In [41]:
product_df_with_lag_final_vf2.show()

+----------+------------+----------+-----------+-------------------+-----------------------------+
|product_id|product_name| sale_date|sale_amount|previous_month_sale|profit_by_over_previous_month|
+----------+------------+----------+-----------+-------------------+-----------------------------+
|         1|      iphone|01-01-2023|    1500000|                  0|                           NA|
|         1|      iphone|01-02-2023|    1300000|            1500000|                       -13.33|
|         1|      iphone|01-03-2023|    1600000|            1300000|                        23.08|
|         1|      iphone|01-04-2023|    1700000|            1600000|                         6.25|
|         1|      iphone|01-05-2023|    1200000|            1700000|                       -29.41|
|         1|      iphone|01-06-2023|    1100000|            1200000|                        -8.33|
|         3|     oneplus|01-01-2023|    1100000|                  0|                           NA|
|         

Unbounded preceeding

In [44]:
product_spec_unbounded_following=Window.partitionBy(col("product_name")).orderBy(col("sale_amount")).\
    rangeBetween(Window.unboundedPreceding,Window.unboundedFollowing)

In [45]:
product_df.withColumn("first_sales_amt",first(col("sale_amount")).over(product_spec_unbounded_following)).\
    withColumn("last_sales_amt",last(col("sale_amount")).over(product_spec_unbounded_following)).\
        show()

+----------+------------+----------+-----------+---------------+--------------+
|product_id|product_name| sale_date|sale_amount|first_sales_amt|last_sales_amt|
+----------+------------+----------+-----------+---------------+--------------+
|         1|      iphone|01-06-2023|    1100000|        1100000|       1700000|
|         1|      iphone|01-05-2023|    1200000|        1100000|       1700000|
|         1|      iphone|01-02-2023|    1300000|        1100000|       1700000|
|         1|      iphone|01-01-2023|    1500000|        1100000|       1700000|
|         1|      iphone|01-03-2023|    1600000|        1100000|       1700000|
|         1|      iphone|01-04-2023|    1700000|        1100000|       1700000|
|         3|     oneplus|01-01-2023|    1100000|        1100000|       1200000|
|         3|     oneplus|01-02-2023|    1120000|        1100000|       1200000|
|         3|     oneplus|01-03-2023|    1160000|        1100000|       1200000|
|         3|     oneplus|01-04-2023|    